#### Install and libraries

In [ ]:
!pip install -q transformers
!pip install -q datasets
!pip install -q apache_beam
# !pip install git+https://github.com/huggingface/transformers.git
!pip install -q huggingface-hub
import os
import re
import json
from datasets import Dataset
import datasets
from collections import Counter
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import transformers
import pyarrow as pa
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, TFAutoModel
import json
from datasets import Dataset
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

     |████████████████████████████████| 4.4 MB 23.1 MB/s 
     |████████████████████████████████| 6.6 MB 41.5 MB/s 
     |████████████████████████████████| 596 kB 47.5 MB/s 
     |████████████████████████████████| 101 kB 11.4 MB/s 
     |████████████████████████████████| 362 kB 18.1 MB/s 
     |████████████████████████████████| 1.1 MB 55.4 MB/s 
     |████████████████████████████████| 212 kB 74.8 MB/s 
     |████████████████████████████████| 140 kB 73.1 MB/s 
     |████████████████████████████████| 127 kB 53.0 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 144 kB 72.8 MB/s 
     |████████████████████████████████| 271 kB 75.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████|

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Functions

### 1. Load JSON

In [ ]:
def load_json(path):
  with open(path, 'r') as json_file:
      with open(path, 'r') as j:
        contents = json.loads(j.read())
      # ('json', data_files='drive/MyDrive/w266/data/natural_questions/data_chunks/761.json')
        df = pd.DataFrame(contents)
        data = Dataset.from_pandas(df)
  return data

### 2. function to add start_byte key to answers.

In [ ]:
def start_btye(examples):
  res = [(ele.start(), ele.end()) for ele in re.finditer(r'\S+', examples['context'])]
  answer_start = examples['answers']['answer_start']
  output = []
  if len(answer_start) != 0:
    for i in answer_start:
      byte_index = res[i][0]
      output.append(byte_index)

  examples['answers']['start_byte'] = output
  return examples

### 3. preprocess_training_examples

In [ ]:
def preprocess_training_examples(examples):
  questions = [q.strip() for q in examples["question"]]
  inputs = tokenizer(
  questions,
  examples["context"],
  max_length=max_length,
  truncation="only_second",
  stride=stride,
  return_overflowing_tokens=True,
  return_offsets_mapping=True,
  padding='max_length'
  )

  offset_mapping = inputs.pop("offset_mapping")
  sample_map = inputs.pop("overflow_to_sample_mapping")
  # offset_mapping = inputs["offset_mapping"]
  # sample_map = inputs["overflow_to_sample_mapping"]
  answers = examples["answers"]
  start_positions = []
  end_positions = []

  for i, offset in enumerate(offset_mapping):   # i is th of generated feature

    sample_idx = sample_map[i]   # sample id in data
    answer = answers[sample_idx]  # original answer dictionary
    
    # Find the start and end of the context
    sequence_ids = inputs.sequence_ids(i)
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    num_answers = len(answer['start_byte'])  # num of answers in each sample
    start =[]
    end = []
    if num_answers == 0:
      pass
    elif num_answers != 0:
      for j in range(num_answers):
        start_char = answer['start_byte'][j]
        end_char = answer['start_byte'][j] + len(answer["text"][j])

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start.append(0)
            end.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start.append(idx - 1)
            # start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end.append(idx + 1)
            # end_positions.append(idx + 1)

    start_positions.append(start)
    end_positions.append(end)
  
  inputs["start_positions"] = start_positions
  inputs["end_positions"] = end_positions
  
  return inputs

### import BERT pretrained model

In [ ]:
from transformers import BertTokenizer, TFBertModel   #AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
bert_model = TFBertModel.from_pretrained('bert-base-cased')
bert_model.trainable = False

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
type(bert_model)

transformers.models.bert.modeling_tf_bert.TFBertModel

In [ ]:
# import datasets
# may want to change this path
train_path = 'drive/MyDrive/w266/data/natural_questions/data_chunks/761.json'
train = load_json(train_path)
train_parsed = train.map(start_btye)

max_length = 300
stride = 50
train_dataset = train_parsed.map(
    preprocess_training_examples,
    batched=True, remove_columns=train_parsed.column_names,
    )
len(train_parsed), len(train_dataset)

  0%|          | 0/148 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

(148, 387)

In [ ]:
len(train_dataset['input_ids'][0])

300

In [ ]:
# may want to change this path
val_path = 'drive/MyDrive/w266/data/natural_questions/data_chunks/1.json'
val = load_json(val_path)
val_parsed = val.map(start_btye)

val_dataset = val_parsed.map(
    preprocess_training_examples,
    batched=True, remove_columns=val_parsed.column_names,
    )
len(val_parsed), len(val_dataset)

  0%|          | 0/200 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

(200, 636)

Model functions:


In [ ]:
def get_shape_list(tensor, expected_rank=None, name=None):
  """Returns a list of the shape of tensor, preferring static dimensions.
  Args:
    tensor: A tf.Tensor object to find the shape of.
    expected_rank: (optional) int. The expected rank of `tensor`. If this is
      specified and the `tensor` has a different rank, and exception will be
      thrown.
    name: Optional name of the tensor for the error message.
  Returns:
    A list of dimensions of the shape of tensor. All static dimensions will
    be returned as python integers, and dynamic dimensions will be returned
    as tf.Tensor scalars.
  """
  if name is None:
    name = tensor.name

  shape = tensor.shape.as_list()

  non_static_indexes = []
  for (index, dim) in enumerate(shape):
    if dim is None:
      non_static_indexes.append(index)

  if not non_static_indexes:
    return shape

  dyn_shape = tf.shape(tensor)
  for index in non_static_indexes:
    shape[index] = dyn_shape[index]
  return shape

## MODELS

### BERT

In [ ]:
def compute_loss(logits, positions, seq_length):
    one_hot_positions = tf.one_hot(
        positions, depth=seq_length, dtype=tf.float32)
    
    log_probs = tf.nn.log_softmax(logits, axis=-1)
    d1, d2 = log_probs.shape
    log_probs = tf.reshape(log_probs, [d1, 1, d2])
    loss = -tf.reduce_mean(
        tf.reduce_sum(one_hot_positions * log_probs, axis=-1))
    return loss

In [ ]:
def create_bertQA(batch_size=32, learning_rate = 0.00005, max_length=300):
    # use pretained BERTmodel 
    bert_model = TFBertModel.from_pretrained('bert-base-cased')

    input_ids = layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
    attention_mask = layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')
    token_type_ids = layers.Input(shape=(max_length,), dtype=tf.int32, name='token_type_ids')

    # only use for loss calculation
    start_positions = layers.Input(shape=(max_length,), dtype=tf.int32, name='start_positions')
    end_positions = layers.Input(shape=(max_length,), dtype=tf.int32, name='end_positions')
    
    # take last_hidden_state to calculate logits for probability of start/end position 
    bert_output = bert_model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    last_hidden_state = bert_output.last_hidden_state  # shape:batch_size, sequence_length, hidden_size

    dense = layers.Dense(2, activation="softmax")(last_hidden_state)

    logits = tf.reshape(dense, [batch_size, last_hidden_state.shape[1], 2])
    logits = tf.transpose(logits, [2, 0, 1])

    unstacked_logits = tf.unstack(logits, axis=0)
    # model outputs
    (start_logits, end_logits) = (unstacked_logits[0], unstacked_logits[1])

    # computing loss
    start_loss = compute_loss(start_logits, start_positions, max_length)
    end_loss = compute_loss(end_logits, end_positions, max_length)
    total_loss = (start_loss + end_loss) / 2.0
    
    model = tf.keras.models.Model(inputs=[input_ids, attention_mask, token_type_ids],
                                  outputs=[start_logits, end_logits])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=total_loss,
                  metrics='accuracy')
    
    return start_logits, end_logits

In [ ]:
tf_train = train_dataset.with_format("tensorflow")
tf_val = val_dataset.with_format("tensorflow")

tf_train = train_dataset.with_format("tensorflow")
tf_val = val_dataset.with_format("tensorflow")

train_features = {x: tf_train[x] for x in ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions' ,'end_positions']}
eval_features = {x: tf_val[x] for x in ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions' ,'end_positions']}
train_labels = {x: tf_train[x] for x in ['start_positions', 'end_positions']}
eval_labels = {x: tf_val[x] for x in ['start_positions', 'end_positions']}

train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
eval_tf_dataset = tf.data.Dataset.from_tensor_slices((eval_features, eval_labels))

# start_positions = tf_train['start_positions'].to_tensor()
# end_positions = tf_train['end_positions'].to_tensor()

In [ ]:
model = create_bertQA(batch_size=32, learning_rate = 0.00005, max_length=300)
outputs = model.fit(x = train_features, y = train_labels, validation_data=eval_tf_dataset, epochs=1)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


TypeError: ignored